In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from utils.model import OpTimeModelProbabilistic

In [ ]:
SWITCH_TIME = 50
NUM_PREFETCHES = 12
NUM_MEMORY_ACCESSES = 10
MEMORY_TIME = 100
IO_TIME_PRE = 2000
IO_TIME_POST = 2000

m = OpTimeModelProbabilistic(SWITCH_TIME, NUM_PREFETCHES)

WIDTH  = 301    # resolution for horizontal axis (latency)
HEIGHT = WIDTH  # resolution for vertical axis (one of other parameters)
STEP = (WIDTH - 1) // 10
VMIN = 40       # throughput degradation in [VMIN, 100]% will be visualized

img = np.zeros((HEIGHT, WIDTH))

In [ ]:
def set_values(start, end, resolution, tick_step, format, denominator):
    values = []
    ticks = []
    for i in range(resolution):
        val = start + (end - start) * (i / (resolution - 1))
        values.append(val)
        if(i % tick_step == 0):
            ticks.append(format % (val / denominator))
    return values, ticks

In [ ]:
def print_percentage(val):
    return '%d%%' % int(val + 0.5)

def plot_heatmap(img, xticks, yticks, ylabel, name):
    img *= 100 / img[:, 0, np.newaxis]
    plt.figure()
    plt.imshow(img, cmap='inferno', interpolation='bilinear', vmin=VMIN, vmax=100)
    b = plt.colorbar(format='%d%%')
    b.ax.tick_params(labelsize=12) 
    c = plt.contour(img, levels=[70, 80, 90, 95, 99, 99.99999],
                    colors='black', alpha=0.9,
                    linewidths=0.5, antialiased=True)
    plt.clabel(c, inline=True, fontsize=12, fmt=print_percentage)
    plt.xlabel('Memory latency $L_{\mathrm{mem}}$ [usec]', fontsize=14, labelpad=10)
    plt.xticks(np.arange(len(xticks)) * STEP, xticks, fontsize=12)
    plt.ylabel(ylabel, fontsize=14, labelpad=10)
    plt.yticks(np.arange(len(yticks)) * STEP, yticks, fontsize=12)
    plt.gca().invert_yaxis()
    plt.savefig(name + '.pdf', bbox_inches='tight')
    plt.show()

In [ ]:
MIN_LATENCY = 0
MAX_LATENCY = 10000
latency_list, xticks = set_values(MIN_LATENCY, MAX_LATENCY, WIDTH, STEP, '%d', 1000)

In [ ]:
MIN_MEMORY_TIME = 0
MAX_MEMORY_TIME = 1000

memory_time_list, yticks = set_values(MIN_MEMORY_TIME, MAX_MEMORY_TIME, HEIGHT, STEP, '%.1f', 1000)
for h, memory_time in enumerate(memory_time_list):
    for w, latency in enumerate(latency_list):
        t = m.reciprocal_throughput(NUM_MEMORY_ACCESSES, memory_time, IO_TIME_PRE, IO_TIME_POST, latency)
        img[h, w] = 1/t

plot_heatmap(img, xticks, yticks, r'Memory suboperation time $T_{\mathrm{mem}}$ [usec]', 'heat_mem_time')

In [ ]:
MIN_IO_TIME_PRE = 0
MAX_IO_TIME_PRE = 10000

io_time_pre_list, yticks = set_values(MIN_IO_TIME_PRE, MAX_IO_TIME_PRE, HEIGHT, STEP, '%.1f', 1000)
for h, io_time_pre in enumerate(io_time_pre_list):
    for w, latency in enumerate(latency_list):
        t = m.reciprocal_throughput(NUM_MEMORY_ACCESSES, MEMORY_TIME, io_time_pre, IO_TIME_POST, latency)
        img[h, w] = 1/t

plot_heatmap(img, xticks, yticks, r'Pre-IO suboperation time $T_{\mathrm{IO}}^{\mathrm{pre}}$ [usec]', 'heat_io_time_pre')

In [ ]:
MIN_IO_TIME_POST = 0
MAX_IO_TIME_POST = 10000

io_time_post_list, yticks = set_values(MIN_IO_TIME_POST, MAX_IO_TIME_POST, HEIGHT, STEP, '%.1f', 1000)
for h, io_time_post in enumerate(io_time_post_list):
    for w, latency in enumerate(latency_list):
        t = m.reciprocal_throughput(NUM_MEMORY_ACCESSES, MEMORY_TIME, IO_TIME_PRE, io_time_post, latency)
        img[h, w] = 1/t

plot_heatmap(img, xticks, yticks, r'Post-IO suboperation time $T_{\mathrm{IO}}^{\mathrm{post}}$ [usec]', 'heat_io_time_post')

In [ ]:
MIN_NUM_MEMORY_ACCESSES = 1
MAX_NUM_MEMORY_ACCESSES = 20

num_memory_accesses_list, yticks = set_values(MIN_NUM_MEMORY_ACCESSES, MAX_NUM_MEMORY_ACCESSES, HEIGHT, STEP, '%d', 1)
for h, num_memory_accesses in enumerate(num_memory_accesses_list):
    for w, latency in enumerate(latency_list):
        t = m.reciprocal_throughput(num_memory_accesses, MEMORY_TIME, IO_TIME_PRE, IO_TIME_POST, latency)
        img[h, w] = 1/t

plot_heatmap(img, xticks, yticks, 'Number $M$ of memory accesses', 'heat_num_memory_accesses')